# Imports

Import python libraries as well as the self written FERMI library.

In [ ]:
import sys, os
from os.path import join, split
from getpass import getuser
from glob import glob
from time import strftime
from tqdm.auto import tqdm
from importlib import reload

# Data
import numpy as np
import xarray as xr
import pandas as pd
import h5py

# Images
import imageio
from imageio import imread

# Plotting
import matplotlib.pyplot as plt
from matplotlib.image import NonUniformImage
import matplotlib.gridspec as gridspec
from matplotlib.path import Path

# pyFAI
import pyFAI
from pyFAI.azimuthalIntegrator import AzimuthalIntegrator
from pyFAI.detectors import Detector

# Self-written libraries
sys.path.append(os.path.abspath(join(os.pardir,"process_FERMI")))
import helper_functions as helper
import mask_lib
import process_FERMI as pf
import interactive
from interactive import cimshow

plt.rcParams["figure.constrained_layout.use"] = True  # replaces plt.tight_layout

In [ ]:
# interactive plotting
import ipywidgets

%matplotlib widget
plt.rcParams["figure.constrained_layout.use"] = True

# Auto formatting of cells
#load_ext jupyter_black

## Functions

In [ ]:
def preprocess_exp(datafolder, extension, keys=None, sort=False, full_rate=False):
    """
    Loads and preprocesses experiment data from a specified datafolder and file extensions (Dark, Only-Laser, Only-FEL, etc).
    
    Parameters:
        datafolder (str): Path to the folder containing the data.
        extension (str): File extension of the data files. (Dark, Only-Laser, Only-FEL, etc. ...)
        keys (list, optional): Specific HDF5-keys to additionaly extract from the data. Defaults to None.
        sort (bool, optional): Whether to sort the data based on a scan axis. Defaults to False.
        full_rate (bool, optional): If True, filters out empty frames before averaging images. Defaults to False.
    
    Returns:
        pandas.DataFrame: Preprocessed experiment data with computed statistics and loaded images.
    """

    
    # Loading experiment data
    print("Loading: %s" % (datafolder + extension))
    exp = pf.get_exp_dataframe(datafolder + extension, keys=keys)
    for k in ["xgm_UH", "xgm_SH", "diode_sum"]:
        exp[k + "_sum"] = exp[k].apply(np.sum)

    exp["diode_sum_mean"] = exp.diode_sum.apply(np.mean)
    exp["diode_sum_sum"] = exp.diode_sum.apply(np.sum)
    exp["diode_sum_std"] = exp.diode_sum.apply(np.std)
    exp["IR_mean"] = exp.IR.apply(np.mean)
    exp["IR_std"] = exp.IR.apply(np.std)
    exp["magnet_mean"] = exp.magnet.apply(np.mean)
    exp["magnet_mean"] = exp.magnet_mean.apply(np.round, args=(3,))
    exp["bunchid"] = exp.bunches.apply(lambda l: l[-1])

    if sort is True:
        exp = exp.sort_values(scan_axis)

    load_images = []
    for idx in range(len(exp["filename"])):
        try:
            if full_rate:
                temp = []
                index = 0
                load_images_full = pf.loadh5(
                    exp["filename"][idx], extra_keys=["DPI/AlignZm", "PAM/FQPDSum"] 
                )[0].astype("float32")
                for i in range(len(load_images_full)):
                    if np.max(load_images_full[i]) > 0:
                        temp.append(load_images_full[i])
                        index += 1
                load_images.append(np.mean(temp, axis=0))
                print("Skipped %d empty frames" % (len(load_images_full) - index))
            else:
                load_images.append(
                    pf.loadh5(
                        exp["filename"][idx], extra_keys=[ "DPI/AlignZm","PAM/FQPDSum"] 
                    )[0].astype("float32")
                )
            print("Loaded %s" % exp["filename"][idx])
        except:
            print("Skipped %s" % exp["filename"][idx])

    exp["images"] = load_images
    
    return exp

In [ ]:
def filter_false_images(images,filter_thres):
    """
    Identifies and filters out inconsistent images based on intensity deviations.
    
    This function computes the mean intensity of each image, compares it to the 
    ensemble median, and filters out images that deviate beyond a threshold 
    determined by the standard deviation.
    
    Parameters:
        images (numpy.ndarray): Array of images to be analyzed.
        filter_thres (float): Threshold for filtering, defining the acceptable 
                              deviation from the median intensity.
    
    Returns:
        numpy.ndarray: Boolean array indicating valid (True) and filtered (False) images.
    """

    # Calc Monitoring parameter
    image_mean = np.nanmean(images,axis = (-2,-1))
    ensemble_mean = np.nanmedian(image_mean)
    image_std = np.nanstd(image_mean)

    # Filter
    valid = np.abs(image_mean-ensemble_mean) < filter_thres * image_std

    # Plot filter condition
    fig, ax = plt.subplots()
    ax.plot(image_mean,'o-')
    ax.grid()
    ax.set_xlabel("Image Index")
    ax.set_ylabel("Image Mean")
    ax.set_title("Check for inconsistencies of the averaged intensity")
    ax.axhline(ensemble_mean,0,images.shape[0],color = 'g',linestyle = '--')
    ax.axhline((ensemble_mean + filter_thres*image_std),0,images.shape[0],color = 'r',linestyle = '--')
    ax.axhline((ensemble_mean - filter_thres*image_std),0,images.shape[0],color = 'r',linestyle = '--')
    
    return valid

# Experimental details

In [ ]:
# Define basic folders
BASEFOLDER = r"/data/beamtimes/FERMI/2310_XPCS"
PROPOSAL = "20224053"
USER = getuser()

In [ ]:
# Dict with most basic experimental parameter
experimental_setup = {
    "px_size": 11e-6,  # pixel_size of camera
    "binning": 1,  # Camera binning
}

# Setup for azimuthal integrator
detector = Detector(
    experimental_setup["binning"] * experimental_setup["px_size"],
    experimental_setup["binning"] * experimental_setup["px_size"],
)

# General saving folder
folder_target = pf.create_folder(join("/data/beamtimes/FERMI/2503_temp_chiral/results", "Ana_Log"))
print("Output Folder: %s" % folder_target)

# Load Data

### Scan

In [ ]:
# Define for loading
sample = "Sample54"
membrane = "I7"  # "E8_PumpedHyst"  # "I2_PumpedHyst"
scan_id = 6
scan = f"%s_%03d" % (membrane, scan_id)

# Is it pumped hysteresis, or static hysteresis?
pumped_hysteresis = False

# Recorded with full rate?
full_rate = False

if pumped_hysteresis:
    # Use last nominal value of the magnet waveform
    scan_axis = "magnet_last"
    # Use FEL+IR as image
    extension = ""
else:
    # Use averaged magnet value
    scan_axis = "magnet_mean"
    # Use OF as image
    extension = "_OF"

# Folder for loading
samplefolder = join(sample, scan)
datafolder = join(BASEFOLDER, samplefolder)
extra_keys = {
    "diode_sum": "PAM/FQPDSum",
    "IR": "Laser/Energy1",
    "magnet": "DPI/CoilCurrent",
    "magnet_waveform": "DPI/CoilWaveform",
    "bunches": "bunches",
    "time": "",
    "samplex": "DPI/SampleX",
    "sampley": "DPI/SampleY",
    "ccdz": "DPI/CcdZ",
}

# Create savefolder
fsave = pf.create_folder(join(folder_target, sample, membrane))

# Loading experiment data
#exp = pf.get_exp_dataframe(datafolder, "_OF", keys=extra_keys)
exp = preprocess_exp(datafolder, extension, keys=extra_keys)
#exp_Saturated = preprocess_exp(datafolder, "_Saturated", keys=extra_keys)

# Add wavelength and distance
experimental_setup["lambda"] = exp["wavelength"][0] * 1e-9
experimental_setup["ccd_dist"] = (exp["ccdz"][0] + 50) * 1e-3

print("Data loaded!")

In [ ]:
# What did you scan?
fig, ax = plt.subplots()
ax.plot(np.arange(len(exp)), exp[scan_axis], "-o")
ax.set_xlabel("Index")
ax.set_ylabel(scan_axis)
ax.grid()

In [ ]:
fig, ax = cimshow(np.stack(exp.images))
ax.set_title("Image Slideshow viewer")

In [ ]:
plt.close('all')

### Dark images

In [ ]:
# Loading experiment data
scan_BG = f"%s_%03d" % (membrane, scan_id)
samplefolder_BG = join(sample, scan_BG)
datafolder_BG = join(BASEFOLDER, samplefolder_BG)

extension = "_BG"

exp_bg = preprocess_exp(datafolder_BG, extension, keys=extra_keys)
exp_bg = exp_bg.sort_values("time")

if pumped_hysteresis:
    dark = np.stack(exp_bg["images"])
else:
    dark = np.mean(np.stack(exp_bg["images"]), axis=0)

print("Data loaded!")

# Plot images
fig, ax = cimshow(dark)
fig.set_size_inches(6, 6)
ax.set_title("Dark Image")

### Laser only

In [ ]:
if pumped_hysteresis:
    # Loading experiment data
    extension = "_OL"
    exp_ol = preprocess_exp(datafolder, extension, keys=extra_keys)
    exp_ol = exp_ol.sort_values("time")

    dark_ol = np.stack(exp_ol["images"])
    print("Data loaded!")

    # Plot images
    fig, ax = cimshow(dark_ol)
    fig.set_size_inches(6, 6)
    ax.set_title("Only Laser Images")
else:
    print("Not executed for static hysteresis!")

### FEL only

In [ ]:
if pumped_hysteresis:
    # Loading experiment data
    extension = "_OF"
    exp_of = preprocess_exp(datafolder, extension, keys=extra_keys)
    exp_of = exp_of.sort_values("time")

    dark_of = np.stack(exp_of["images"])
    print("Data loaded!")

    # Plot images
    fig, ax = cimshow(dark_of)
    fig.set_size_inches(6, 6)
    ax.set_title("Only FEL Images")
else:
    print("Not executed for static hysteresis!")

### Subtract dark images (for dynamic hysteresis also OL, OF) and normalize images to I0

In [ ]:
# Incident intensity
fig, ax = plt.subplots()
ax.plot(np.arange(len(exp)), exp["diode_sum_sum"])
ax.set_xlabel("Image Index")
ax.set_ylabel("Incident Intensity (a.u.)")

In [ ]:
# Which key to use for normalization?
norm_key = "diode_sum_sum"
filter_key = "diode_sum_sum"

if pumped_hysteresis:
    # Loop over images
    images = []
    images_of = []
    for index, r in tqdm(exp.iterrows(), total=len(exp)):
        # Find closest dark image in time series
        idx = np.argmin(abs(r.time - exp_bg.time))
        im_bg = exp_bg.iloc[idx]["images"]

        # Find closest only laser image in time series
        idx = np.argmin(abs(r.time - exp_ol.time))
        im_ol = exp_ol.iloc[idx]["images"]

        # Find closest only fel image in time series
        idx = np.argmin(abs(r.time - exp_of.time))
        im_of = exp_of.iloc[idx]["images"]

        # Subtract background
        im = (r.images - im_ol) / r[norm_key]
        of_norm = (im_of - im_bg) / exp_of.iloc[idx][norm_key]
        im = im - of_norm

        images.append(im)
        images_of.append(of_norm)
else:
    images = np.stack(exp.images) - dark
    images = images / np.broadcast_to(np.array(exp["diode_sum_mean"]), images.T.shape).T

im_mean = np.mean(images, axis=0)

In [ ]:
fig, ax = cimshow(images)
ax.set_title("Normalized and Background corrected")

## Draw beamstop mask

In [ ]:
poly_mask = interactive.draw_polygon_mask(im_mean)

In [ ]:
# Take poly coordinates and mask from widget
p_coord = poly_mask.get_vertice_coordinates()
mask = poly_mask.full_mask.astype(int)

cimshow(mask)

print("Mask coordinates: %s" % p_coord)

In [ ]:
def load_poly_coordinates():
    """
    Dictionary that stores polygon corner coordinates of all drawn masks
    Example: How to add masks with name "test":
    mask_coordinates["test"] = copy coordinates from above
    """

    # Setup dictonary
    mask_coordinates = dict()

    mask_coordinates["bs_cross"] = [[(970.6033322491838, 1035.7034550617395), (527.5411796884623, 1019.5333035084286), (12.617148197288515, 1002.8622019364727), (-60.139858829924265, 994.1121885848413), (-26.96853342189422, 1162.7330927423272), (529.1581948437935, 1182.8518341968697), (964.1352716278594, 1195.7879554395186), (968.4083162201969, 1259.459762784269), (924.0397667896182, 2057.6005935990224), (1087.1331264408084, 2070.6480623711177), (1113.3401451709547, 1599.6803482278478), (1133.0411391151633, 1201.8382747710389), (1585.701758243843, 1220.5306473819905), (2083.078220662558, 1245.3994705029263), (2063.7358026796082, 1085.1337215013402), (1505.5688837430498, 1057.5016958114118), (1135.2997394980064, 1040.9224803974546), (1175.0258546913572, -7.439510570785615), (1012.0272730816591, -7.439510570785842), (977.3530293574142, 874.0864548802089)]]
    mask_coordinates["membranes"] = [[(924.3859891002303, 989.4662819473956), (925.461764876891, 1047.4386478049537), (993.2356388065143, 1031.4215372371623), (981.4021052632468, 984.0874030640921)], [(1081.4492524926907, 992.5740831952585), (1084.676579822673, 1040.9839931449894), (1132.0107139957431, 1046.362872028293), (1166.435538848885, 1017.3169260584543), (1162.1324357422423, 982.8921012053124), (1119.1014046758148, 985.0436527586337)], [(1205.1634668086697, 986.1194285352944), (1175.0417450621705, 1003.3318409618654), (1184.7237270521168, 1033.4535627083646), (1222.3758792352407, 1033.4535627083646), (1249.270273651758, 1009.7864956218295), (1250.3460494284186, 988.2709800886157)], [(1276.1646680682752, 993.6498589719192), (1276.1646680682752, 1030.2262353783826), (1303.0590624847923, 1035.6051142616861), (1333.1807842312915, 1031.3020111550431), (1333.1807842312915, 993.6498589719192)], [(1364.3782817544516, 996.8771863019012), (1365.454057531112, 1035.6051142616861), (1403.1062097142362, 1037.7566658150074), (1420.3186221408073, 1033.4535627083646), (1414.9397432575038, 991.4983074185978)]]
    return mask_coordinates

In [ ]:
# Which drawn masks do you want to load?
polygon_names = ["bs_cross","membranes"] 
mask = mask_lib.load_poly_masks(images[0].shape,load_poly_coordinates(),polygon_names)

fig, ax = plt.subplots(1, 2, figsize=(10, 5), sharex=True, sharey=True)
mi, ma = np.percentile(im_mean, [1, 99])
ax[0].imshow(im_mean * (1 - mask), vmin=mi, vmax=ma)
ax[0].set_title("(1-mask)")
ax[1].imshow(im_mean * mask, vmin=mi, vmax=ma)
ax[1].set_title("mask")

## Find center

### Basic widget to find center

Try to **align** the circles to the **center of the scattering pattern**. Care! Position of beamstop might be misleading and not represent the actual center of the hologram. 

In [ ]:
# Set center position via widget
ic = interactive.InteractiveCenter(images,c0=1098,c1 = 1033,rBS=95)

In [ ]:
# Get center positions
center = [ic.c0, ic.c1]
print(f"Center:", center)

### Azimuthal integrator widget for finetuning

In [ ]:
# Setup azimuthal integrator for virtual geometry
ai = interactive.AzimuthalIntegrator(
    dist=experimental_setup["ccd_dist"],
    detector=detector,
    wavelength=experimental_setup["lambda"],
    poni1=center[0]
    * experimental_setup["px_size"]
    * experimental_setup["binning"],  # y (vertical)
    poni2=center[1]
    * experimental_setup["px_size"]
    * experimental_setup["binning"],  # x (horizontal)
)

In [ ]:
# Plotting to find  relevant q range
I_t, q_t, phi_t = ai.integrate2d(
    im_mean,
    200,
    radial_range=(0, 0.05),
    unit="q_nm^-1",
    correctSolidAngle=False,
    dummy=np.nan,
    mask=mask,
    method="bbox"
)
az2d = xr.DataArray(I_t, dims=("phi", "q"), coords={"q": q_t, "phi": phi_t})

# Plot
fig, ax = plt.subplots()
mi, ma = np.nanpercentile(I_t, [1, 99])
az2d.plot.imshow(ax=ax, vmin=mi, vmax=ma)
plt.title(f"Azimuthal integration")

# Vertical lines
# q_lines = [0.025, 0.05]
# for qt in q_lines:
#    ax.axvline(qt, ymin=0, ymax=180, c="red")

In [ ]:
aic = interactive.AzimuthalIntegrationCenter(
    # np.log10(images[36] - np.min(images[36]) + 1),
    im_mean,
    ai,
    c0=center[0],
    c1=center[1],
    mask=mask,
    im_data_range=[1, 95],
    radial_range=(0.00, 0.05),
    qlines=[125, 145],
)

In [ ]:
# Get center positions
center = [aic.c0, aic.c1]
print(f"Center:", center)

# Azimuthal Integration

In [ ]:
# Setup final azimuthal integrator for virtual geometry
ai = interactive.AzimuthalIntegrator(
    dist=experimental_setup["ccd_dist"],
    detector=detector,
    wavelength=experimental_setup["lambda"],
    poni1=center[0]
    * experimental_setup["px_size"]
    * experimental_setup["binning"],  # y (vertical)
    poni2=center[1]
    * experimental_setup["px_size"]
    * experimental_setup["binning"],  # x (horizontal)
)

In [ ]:
# Do 2d Azimuthal integration of all images and add to xarray
list_i2d = []
for im in tqdm(images):
    i2d, q, chi = ai.integrate2d(im, 500, 90, dummy=np.nan, mask=mask, method="bbox")
    list_i2d.append(i2d)

# Setup xarray
data = xr.Dataset()
data["images"] = xr.DataArray(images, dims=["index", "y", "x"])
data[scan_axis] = xr.DataArray(exp[scan_axis], dims=["index"])
data["q"] = q
data["chi"] = chi
data["i2d"] = xr.DataArray(list_i2d, dims=["index", "chi", "q"])
data = data.assign_attrs({"center": center})

# If it's a pumped hysteresis, do it also for the OF case
if pumped_hysteresis:
    list_i2d_of = []
    for im in tqdm(images_of):
        i2d, q, chi = ai.integrate2d(im, 500, 90, dummy=np.nan, mask=mask)
        list_i2d_of.append(i2d)

    # Setup xarray
    data["images_of"] = xr.DataArray(images_of, dims=["index", "y", "x"])
    data["i2d_of"] = xr.DataArray(list_i2d_of, dims=["index", "chi", "q"])

## Select relevant chi-range

In [ ]:
# Plot 2d and 1d azimuthal integration to estimate the relevant chi and q range
# which image to show?
idx = 0

# Which chi-mode? (all,hetero,homo)
chi_mode = "all"

# Select chi-range
if chi_mode == "all":
    sel_chi = (data.chi <= 180) * (data.chi >= -180)
    data["i1d"] = data.i2d.where(sel_chi, drop=True).mean("chi")
    if pumped_hysteresis:
        data["i1d_of"] = data.i2d_of.where(sel_chi, drop=True).mean("chi")
elif chi_mode == "hetero":
    sel_chi = (data.chi < 180) * (data.chi > 95)
    data["i1d"] = data.i2d.where(sel_chi, drop=True).mean("chi")
    if pumped_hysteresis:
        data["i1d_of"] = data.i2d_of.where(sel_chi, drop=True).mean("chi")
elif chi_mode == "homo":
    sel_chi = (data.chi <= -95) * (data.chi >= -180) + (data.chi <= 90) * (
        data.chi >= 5
    )
    data["i1d"] = data.i2d.where(sel_chi, drop=True).mean("chi")
    if pumped_hysteresis:
        data["i1d_of"] = data.i2d_of.where(sel_chi, drop=True).mean("chi")

# Plot
fig, ax = plt.subplots(
    2,
    1,
    figsize=(8, 8),
    sharex=True,
)
mi, ma = np.nanpercentile(I_t, [0.1, 90])
data["i2d"][idx].plot.imshow(ax=ax[0], vmin=mi, vmax=ma)
ax[0].set_title(f"2d Azimuthal integration")
ax[0].grid()

# Plot 1d azimuthal integration to estimate the relevant q-range
ax[1].plot(data.q, data.i1d[idx])
ax[1].set_yscale("log")
ax[1].set_title("1d Azimuthal Integration")
ax[1].grid()
ax[1].set_ylabel("Integrated intensity")
ax[1].set_xlabel("q")

## Select relevant q-range

In [ ]:
# Select relevant q-range for averaging
q0, q1 = 0.01, 0.04
binning = False
bins = []

# Get SAXS from q-range
sel = (data.q > q0) * (data.q < q1)
data["saxs"] = data.i1d.where(sel, drop=True).mean("q")
if pumped_hysteresis:
    data["saxs_of"] = data.i1d_of.where(sel, drop=True).mean("q")

# Averaging of same scan axis values or binning
if binning is True:
    # Execute binning
    data_bin = data.groupby_bins(scan_axis, bins).mean()

    # Rename binned values, drop intervals as those cannot be save in h5
    bin_scan_axis = scan_axis + "_bins"
    data_bin = data_bin.swap_dims({bin_scan_axis: scan_axis})
    data_bin = data_bin.drop(bin_scan_axis)
else:
    _, count = np.unique(data[scan_axis].values, return_counts=True)
    if np.any(count > 1):
        data_bin = data.groupby(scan_axis).mean()
    else:
        data_bin = data.swap_dims({"index": scan_axis})

# To create log plot
data_bin["i1dlog"] = np.log10(data_bin["i1d"]+ 1)

# Add scan identifier
data_bin["scan"] = scan

# Add AI mask
data_bin["mask"] = xr.DataArray(mask, dims=["y", "x"])

# Direction of "time"
if np.sum(data[scan_axis][1:].values - data[scan_axis][0:-1].values) >= 0:
    data_bin["order"] = 1
elif np.sum(data[scan_axis][1:].values - data[scan_axis][0:-1].values) < 0:
    data_bin["order"] = -1

# Plot
if pumped_hysteresis:
    label_set = "pump effect (IM-OF-OL)"
else:
    label_set = "only FEL (static)"
fig, ax = plt.subplots()
ax.plot(
    data_bin[scan_axis].values,
    data_bin["saxs"].values,
    "o-",
    label=label_set,
)
if pumped_hysteresis:
    ax.plot(
        data_bin[scan_axis].values,
        data_bin["saxs_of"].values,
        "o-",
        label="only FEL (OF)",
    )
# ax.plot(
#    data_bin[scan_axis].values,
#    np.mean(data_bin["images"].values * (1 - mask), axis=(1, 2)),
#    label="Simple Mean",
# )

ax.set_xlabel(scan_axis)
ax.set_ylabel("Integrated SAXS")
ax.set_title("Scan: %s (Azimuthal Integration)" % scan)
ax.grid()
ax.legend()

# Save fig
fname = join(fsave, "Hysteresis_%s_%s_%s.png" % (scan, chi_mode, USER))
print("Saving: %s" % fname)
plt.savefig(fname)

# Hysteresis Plot

## Select roi for plotting

How to use:
1. Zoom into the image and adjust your FOV until you are satisfied.
2. Save the axes coordinates.

In [ ]:
fig, ax = cimshow(data_bin["images"].values)

In [ ]:
# Takes start and end of x and y axis
x1, x2 = ax.get_xlim()
y2, y1 = ax.get_ylim()
roi = np.array([int(y1), int(y2), int(x1), int(x2)])
roi_s = np.s_[roi[0] : roi[1], roi[2] : roi[3]]
print(f"Roi:", roi)

## Plotting

In [ ]:
# Find max and min considering all images
allmin, allmax = np.nanpercentile(data_bin["i2d"].values, [1, 99.9])
# allmin, allmax = np.nanpercentile(images - images[0], [3, 97])

if allmin < .1:
    allmin = .1

print("Min: %d Max: %d" % (allmin, allmax))

# Create folder for gif single frames
folder_gif = helper.create_folder(join(fsave, "Scan_%s" % scan))

im_fnames = []
for i in tqdm(range(len(data_bin[scan_axis].values))):
    # Plot for averaged image
    fig = plt.figure(figsize=(6, 10))
    gs1 = gridspec.GridSpec(
        4,
        1,
        figure=fig,
        left=0.2,
        bottom=0.05,
        right=0.975,
        top=1.1,
        wspace=0,
        hspace=0,
        height_ratios=[6, 1, 2, 1],
    )

    # Plot image roi
    ax0 = fig.add_subplot(gs1[0])
    m = ax0.imshow(data_bin["images"][i].values[roi_s]*(1-mask[roi_s]), vmin=allmin, vmax=allmax)
    plt.colorbar(m, ax=ax0, pad=0.045, location="bottom")

    # Plot 1d azimuthal integration
    ax1 = fig.add_subplot(gs1[1])
    tmp = data_bin.i1d[i]
    ax1.plot(data_bin.q, tmp)
    ax1.set_xlabel("q")
    ax1.set_ylabel("Mean Intensity")
    ax1.set_xlim([q0, q1])
    ax1.set_ylim([allmin, allmax])
    ax1.set_yscale("log")
    ax1.grid()
    
    ax2 = fig.add_subplot(gs1[2])
    vmin, vmax = np.nanpercentile(data_bin["i1dlog"], [.1, 99.9])
    data_bin["i1dlog"].plot.contourf(
        x=scan_axis,
        y="q",
        ax=ax2,
        cmap="viridis",
        add_colorbar=False,
        vmin=vmin,
        vmax=vmax,
        levels=200,
        ylim = [q0,q1]
    )
    ax2.vlines(data_bin[scan_axis].values[i], q0, q1,'r')
    ax2.hlines(q0, data_bin[scan_axis].min(),data_bin[scan_axis].max(),'w',linestyles='dashed')
    ax2.hlines(q1, data_bin[scan_axis].min(),data_bin[scan_axis].max(),'w',linestyles='dashed')

    # Plot SAXS Intensity
    ax3 = fig.add_subplot(gs1[3])
    ax3.plot(data_bin[scan_axis].values, data_bin["saxs"].values)
    ax3.scatter(data_bin[scan_axis].values[i], data_bin["saxs"].values[i], 20, color="r")
    ax3.set_xlabel(scan_axis)
    ax3.set_ylabel("Mean intensity")
    ax3.grid()
    ax3.set_xlim(data_bin[scan_axis].min(),data_bin[scan_axis].max())

    # Title and fname
    ax0.set_title(f"%s:  %s = %s" % (scan, scan_axis, data_bin[scan_axis].values[i]))

    # Save
    fname = join(folder_gif, "Hysteresis_%s_%s_%03d_%s.png" % (scan, chi_mode, i, USER))
    im_fnames.append(fname)
    plt.savefig(fname)
    plt.close()


# Create gif for 1d AI
fname = f"SAXS_%s_%s_%s.gif" % (scan, chi_mode, USER)
gif_path = join(fsave, fname)
print("Saving gif:%s" % gif_path)
helper.create_gif(im_fnames,gif_path,fps=2)
print("Done!")

In [ ]:
# Drop images
data_bin2 = data_bin.drop_vars(["images"])

# Save log
folder = join(fsave, "Logs")
helper.create_folder(folder)
fname = join(folder, "Log_Hysteresis_Scan_%03d_%s_%s.nc" % (scan_id, chi_mode, USER))

print(f"Saving:", fname)
data_bin2.to_netcdf(fname)

# Batch processing

## Loading and pre-processing

In [ ]:
def worker(
    samplefolder,
    scan,
    mask,
    ai,
    scan_axis,
    binning=None,
    keys=None,
    sort=False,
):
    # Load scan data
    datafolder = join(BASEFOLDER, samplefolder)
    exp = preprocess_exp(datafolder, "_OF", keys=keys, sort=sort)

    # Load background images
    exp_bg = preprocess_exp(datafolder, "_BG", keys=keys)

    # Normalize images
    dark = np.mean(np.stack(exp_bg["images"]), axis=0)
    images = np.stack(exp.images) - dark
    images = images / np.broadcast_to(np.array(exp["diode_sum_mean"]), images.T.shape).T

    # Create xarray dataset
    data = xr.Dataset()
    data["images"] = xr.DataArray(images, dims=["index", "y", "x"])
    data[scan_axis] = xr.DataArray(exp[scan_axis], dims=["index"])

    # Do 2d Azimuthal integration of all images and append them to list
    list_i2d = []
    for im in tqdm(data["images"].values):
        i2d, q, chi = ai.integrate2d(im, 500, 90, dummy=np.nan, mask=mask,method="bbox")
        list_i2d.append(i2d)

    # Add to xarray
    data["q"] = q
    data["chi"] = chi
    data["i2d"] = xr.DataArray(list_i2d, dims=["index", "chi", "q"])
    data["i1d"] = data.i2d.where(sel_chi, drop=True).mean("chi")

    # Averaging of same scan axis values or binning
    if binning is None:
        _, count = np.unique(data[scan_axis].values, return_counts=True)
        if np.any(count > 1):
            data_bin = data.groupby(scan_axis).mean()
        else:
            data_bin = data.swap_dims({"index": scan_axis})

    else:
        # Execute binning
        data_bin = data.groupby_bins(scan_axis, bins).mean()

        # Rename binned values, drop intervals as those cannot be save in h5
        bin_scan_axis = scan_axis + "_bins"
        data_bin = data_bin.swap_dims({bin_scan_axis: scan_axis})
        data_bin = data_bin.drop(bin_scan_axis)

    # Add log plot
    data_bin["i1dlog"] = np.log10(data_bin["i1d"]+ 1)
    
    # Add AI mask
    data_bin["mask"] = xr.DataArray(mask, dims=["y", "x"])

    # Add file scan labels
    data_bin["scan"] = scan

    # Direction of "time"
    if np.sum(data[scan_axis][1:].values - data[scan_axis][0:-1].values) >= 0:
        data_bin["order"] = 1
    elif np.sum(data[scan_axis][1:].values - data[scan_axis][0:-1].values) < 0:
        data_bin["order"] = -1


    # Drop images to save disk space
    data_save = data_bin.drop_vars(["images"])

    # Save log
    folder = join(fsave, "Logs")
    fname = join(folder, "Log_Hysteresis_Scan_%s_%s_%s.nc" % (scan, chi_mode, USER))
    print(f"Saving:", fname)
    data_save.to_netcdf(fname)

    return data_bin

In [ ]:
# Name of scans (see pad)
scans = [
    "I7_006",
    "I7_007",
    "I7_008",
    "I7_010",
]

# Analysis options
bins = []

# Setup xarray for scans
data_scans = []

# Loop over scans
for scan in tqdm(scans):
    # Process data
    samplefolder = join(sample, scan)
    data = worker(
        samplefolder,
        scan,
        mask,
        ai,
        scan_axis,
        binning=None,
        keys=extra_keys,
        sort=False,
    )

    # Add to xarray list
    data_scans.append(data)

# Combine separate xarrays
data_scans = xr.concat(data_scans, dim="scanid")
data_scans

## Calc and plot SAXS

In [ ]:
# Do you want to norm the hysteresis?
normalization = True

# Select relevant q-range for averaging
# q0, q1 = 0.003, 0.04

# Get SAXS from q-range
sel = (data_scans.q > q0) * (data_scans.q < q1)
data_scans["saxs"] = data_scans.i1d.where(sel, drop=True).mean("q")

# Plot all SAXS images individually
for scanid in data_scans["scanid"].values:
    fig, ax = plt.subplots()

    # Normalize?
    if normalization is True:
        tmp_data = data_scans["saxs"][scanid].values
        tmp_data = norm(data_scans["saxs"][scanid].values[1:])
    else:
        tmp_data = data_scans["saxs"][scanid].values[1:]

    ax.plot(data_scans[scan_axis].values[1:], tmp_data, "o-")
    ax.set_xlabel(scan_axis)
    ax.set_ylabel("Integrated SAXS")
    ax.set_title("Scan: %s" % data_scans["scan"][scanid].values)
    ax.grid()

    # Save fig
    fname = join(
        fsave,
        "Hysteresis_%s_%s_%s.png" % (data_scans["scan"][scanid].values, chi_mode, USER),
    )
    print("Saving: %s" % fname)
    plt.savefig(fname)
    plt.close()

# Plot them together
fig, ax = plt.subplots()
for scanid in data_scans["scanid"].values:
    # Normalize?
    if normalization is True:
        tmp_data = data_scans["saxs"][scanid].values
        tmp_data = norm(tmp_data[1:])
    else:
        tmp_data = data_scans["saxs"][scanid].values[1:]
    ax.plot(
        data_scans[scan_axis].values[1:],
        tmp_data,
        "o-",
        label=data_scans["scan"][scanid].values,
    )

ax.set_xlabel(scan_axis)
ax.set_ylabel("Integrated SAXS")
ax.grid()
ax.legend()

# Save fig
fname = join(
    fsave,
    "Hysteresis_%s_%s_%s_%s.png"
    % (data_scans["scan"][0].values, data_scans["scan"][-1].values, chi_mode, USER),
)
plt.savefig(fname)

In [ ]:
# Export gif for all scans individually
for scan_id in tqdm(data_scans["scanid"].values):
    tmp_xr = data_scans.sel(scanid = scan_id)
    
    if tmp_xr["order"] == 1:
        tmp_xr = tmp_xr.reindex(magnet_mean=data_scans.magnet_mean[::-1])

    # Find max and min considering all images
    allmin, allmax = np.nanpercentile(tmp_xr["i2d"].values, [1, 99])
    if allmin < .1:
        allmin = .1
    print("Min: %d Max: %d" % (allmin, allmax))

    # Create folder for gif single frames
    folder_gif = helper.create_folder(join(fsave, "Scan_%s" % tmp_xr["scan"].values))

    # Normalize?
    if normalization is True:
        tmp_data = tmp_xr["saxs"].values
        tmp_data = norm(tmp_data[1:])
    else:
        tmp_data = tmp_xr["saxs"].values
        tmp_data = tmp_data[1:]

    im_fnames = []
    for i in tqdm(range(1, len(tmp_xr[scan_axis].values))):
        # Plot for averaged image
        fig = plt.figure(figsize=(6, 10))
        gs1 = gridspec.GridSpec(
            4,
            1,
            figure=fig,
            left=0.2,
            bottom=0.05,
            right=0.975,
            top=1.1,
            wspace=0,
            hspace=0,
            height_ratios=[6, 1, 2, 1],
        )
    
        # Plot image roi
        ax0 = fig.add_subplot(gs1[0])
        m = ax0.imshow(tmp_xr["images"][i].values[roi_s]*(1-mask[roi_s]), vmin=allmin, vmax=allmax)
        plt.colorbar(m, ax=ax0, pad=0.045, location="bottom")
    
        # Plot 1d azimuthal integration
        ax1 = fig.add_subplot(gs1[1])
        tmp = tmp_xr.i1d[i]
        ax1.plot(tmp_xr.q, tmp)
        ax1.set_xlabel("q")
        ax1.set_ylabel("Mean Intensity")
        ax1.set_xlim([q0, q1])
        ax1.set_ylim([allmin, allmax])
        ax1.set_yscale("log")
        ax1.grid()
        
        ax2 = fig.add_subplot(gs1[2])
        vmin, vmax = np.nanpercentile(tmp_xr["i1dlog"], [1, 99])
        tmp_xr["i1dlog"].plot.contourf(
            x=scan_axis,
            y="q",
            ax=ax2,
            cmap="viridis",
            add_colorbar=False,
            vmin=vmin,
            vmax=vmax,
            levels=300,
            ylim = [q0,q1]
        )
        ax2.vlines(tmp_xr[scan_axis].values[i], q0, q1,'r')
        ax2.hlines(q0, tmp_xr[scan_axis].min(),data_bin[scan_axis].max(),'w',linestyles='dashed')
        ax2.hlines(q1, tmp_xr[scan_axis].min(),data_bin[scan_axis].max(),'w',linestyles='dashed')
    
        # Plot SAXS Intensity
        ax3 = fig.add_subplot(gs1[3])
        ax3.plot(tmp_xr[scan_axis].values, tmp_xr["saxs"].values)
        ax3.scatter(tmp_xr[scan_axis].values[i], tmp_xr["saxs"].values[i], 20, color="r")
        ax3.set_xlabel(scan_axis)
        ax3.set_ylabel("Mean intensity")
        ax3.grid()
        ax3.set_xlim(tmp_xr[scan_axis].min(),data_bin[scan_axis].max())
    
        # Title and fname
        ax0.set_title(f"%s:  %s = %s" % (tmp_xr["scan"].values, scan_axis, tmp_xr[scan_axis].values[i]))

        # Save
        fname = join(
            folder_gif,
            "Hysteresis_%s_%03d_%s_%s.png"
            % (tmp_xr["scan"].values, i, chi_mode, USER),
        )
        im_fnames.append(fname)
        plt.savefig(fname)
        plt.close()

    # Create gif for 1d AI
    var = [imageio.imread(file) for file in im_fnames]
    fname = f"Hysteresis_%s_%s_%s.gif" % (
        tmp_xr["scan"].values,
        chi_mode,
        USER,
    )
    gif_path = join(fsave, fname)
    print("Saving gif:%s" % gif_path)
    helper.create_gif(im_fnames,gif_path,fps=2)
    print("Done!")

In [ ]:
plt.close("all")